
# Messages from the Mayor Analysis

Data is harvest/constructed in this [notebook](https://github.com/BrockDSL/ARCH_Data_Explore/blob/main/Junkin_Processing.ipynb)

Originally found here: [https://www.pelham.ca/en/living-here/covid-19-messages-from-the-mayor.aspx](https://www.pelham.ca/en/living-here/covid-19-messages-from-the-mayor.aspx)

Now deleted. April 21, 2021 last [snapshot](https://web.archive.org/web/20210422114625/https://www.pelham.ca/en/living-here/covid-19-messages-from-the-mayor.aspx)


In [2]:
!pip install flair

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 401 kB 13.6 MB/s 
     |████████████████████████████████| 981 kB 41.2 MB/s 
     |████████████████████████████████| 53 kB 499 kB/s 
     |████████████████████████████████| 19.7 MB 46.2 MB/s 
     |████████████████████████████████| 1.6 MB 11.3 MB/s 
     |████████████████████████████████| 182 kB 31.6 MB/s 
     |████████████████████████████████| 1.2 MB 43.2 MB/s 
     |████████████████████████████████| 46 kB 2.3 MB/s 
     |████████████████████████████████| 788 kB 36.2 MB/s 
     |████████████████████████████████| 5.5 MB 39.7 MB/s 
     |████████████████████████████████| 200 kB 62.4 MB/s 
     |████████████████████████████████| 62 kB 1.6 MB/s 
     |████████████████████████████████| 7.6 MB 33.2 MB/s 
  Created wheel for mpld3: filename=mpld3-0.3-py3-none-any.whl size=116702 sha256=02df921d49faa4cbe7079301952f917973994d55869926994edf8d33c6f12064
  Sto

In [3]:
from ipywidgets import interact
import matplotlib.pyplot as plt
import pandas as pd
from textblob import TextBlob
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

# Import flair pre-trained sentiment model
from flair.models import TextClassifier
classifier = TextClassifier.load('en-sentiment')
classifier_rnn_fast = TextClassifier.load('sentiment-fast')

# Import flair Sentence to process input text
from flair.data import Sentence
import requests
import matplotlib.pyplot as plt
pd.set_option('display.max_colwidth', 999)

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


2022-11-28 03:18:14,278 https://nlp.informatik.hu-berlin.de/resources/models/sentiment-curated-distilbert/sentiment-en-mix-distillbert_4.pt not found in cache, downloading to /tmp/tmpdey_vl4h


100%|██████████| 265512723/265512723 [00:09<00:00, 27316730.07B/s]

2022-11-28 03:18:24,324 copying /tmp/tmpdey_vl4h to cache at /root/.flair/models/sentiment-en-mix-distillbert_4.pt


2022-11-28 03:18:24,854 removing temp file /tmp/tmpdey_vl4h
2022-11-28 03:18:24,908 loading file /root/.flair/models/sentiment-en-mix-distillbert_4.pt


...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

2022-11-28 03:18:27,923 https://nlp.informatik.hu-berlin.de/resources/models/sentiment-curated-fasttext-rnn/sentiment-en-mix-ft-rnn_v8.pt not found in cache, downloading to /tmp/tmpg_1bepyw


100%|██████████| 1241977025/1241977025 [00:48<00:00, 25413864.45B/s]

2022-11-28 03:19:17,157 copying /tmp/tmpg_1bepyw to cache at /root/.flair/models/sentiment-en-mix-ft-rnn_v8.pt


2022-11-28 03:19:21,756 removing temp file /tmp/tmpg_1bepyw
2022-11-28 03:19:21,940 loading file /root/.flair/models/sentiment-en-mix-ft-rnn_v8.pt


In [4]:
junkin_data = pd.read_csv("https://raw.githubusercontent.com/BrockDSL/ARCH_Data_Explore/main/Junkin_Posts.csv")
junkin_data['Date'] = pd.to_datetime(junkin_data['Date'])

In [5]:
junkin_data.head(1)

,Date,Entry
0,2021-04-20,"On Friday the Provincial government introduced new measures to try to halt the runaway third wave of COVID variants that are the toughest in North America. Among other directives, police and bylaw officers were given the authority to stop residents who were either driving or walking, and ask them their reason for be- ing outside their residences. After almost all police forces in the province stated they would not be stopping cars to do these checks, the Provincial government backtracked.During his Friday announcement, Premier Ford said, “We, as a group, are not saying don’t take a walk around and get some exercise,” but added there needed to be measures in place that target large groups of people who meet in public parks without social distancing. Following this announcement, Pelham’s playgrounds like all other playgrounds located in Ontario were off limits. Many experts did not agree with the closing of the playgrounds because infections from surface con- tact are thought to be ..."


In [6]:
#Sentiment score from the classifiers is always a positive number indicating probability of the pred={POSITIVE,NEGATIVE}
#Treat the score negative if pred=NEGATIVE and treat the score as a % of positivity/negativity
def rescale_sentiment(sentence):
  score = sentence.labels[0].score
  pred = sentence.labels[0].value
  
  if pred == 'POSITIVE': return score
  else: return score * -1

def flair_sentiment(texts, classifier):
    sentences = [Sentence(text) for text in texts]
    classifier.predict(sentences, mini_batch_size=32, verbose=True)
    return [rescale_sentiment(sent) for sent in sentences]

In [7]:
junkin_data['vader_comp'] = junkin_data.apply(lambda row: sid.polarity_scores(row.Entry)['compound'], axis=1)
junkin_data['text_blob'] = junkin_data.apply(lambda row: TextBlob(row.Entry).sentiment.polarity, axis=1)
junkin_data['bert'] = flair_sentiment(junkin_data.Entry, classifier)
junkin_data['rnn'] = flair_sentiment(junkin_data.Entry, classifier_rnn_fast)


Batch inference: 100%|██████████| 2/2 [00:01<00:00,  1.42it/s]


In [8]:
algorithms = ['vader_comp','text_blob','bert','rnn']

def plot_sentiment(algorithm):
    
    fig, ax = plt.subplots(figsize=(12,8))

    ax.bar(junkin_data["Date"],junkin_data[algorithm])
    ax.set_title("Sentiment of 'Messages from the Mayor'")
    ax.set_ylabel("Sentiment")
    ax.set_xlabel("Date")


interact(plot_sentiment, algorithm=algorithms)

interactive(children=(Dropdown(description='algorithm', options=('vader_comp', 'text_blob', 'bert', 'rnn'), va…

<function __main__.plot_sentiment(algorithm)>